In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("MetroPT2.csv")

In [3]:
df.columns

Index(['timestamp', 'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs',
       'Oil_temperature', 'Flowmeter', 'Motor_current', 'COMP', 'DV_eletric',
       'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level',
       'Caudal_impulses', 'gpsLat', 'gpsLong', 'gpsSpeed', 'gpsQuality'],
      dtype='object')

In [5]:
df.timestamp.unique()

KeyboardInterrupt: 

In [7]:
df.shape

(7116940, 21)